# Synthethischer Datensatz

**Statische Variablen**: age, height, colors

**Dynamische Variablen**: brand


Anzahl an Usern kann durch n_user festgelegt werden und Länge der Zeitreihe durch repetitions.

Für Target Variable kann eigene Logik implementiert werden. Aktuelle Logik: wenn Adidas direkt nach Nike auftritt, dann 1, sonst 0.  

Weiter unten wird nach Usern gesucht, wo genau die letzte Reihe (welche zur prediction verwendet wird) adidas beinhaltet und die Reihe davor Nike. Logischerweise, sollte durch das wegnehmen von Nike ein großer Unterschied bei den predicted probabilities entstehen. 

In [389]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import lightning.pytorch as pl

import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import CrossEntropy

from torch.nn import Softmax

import shap

In [390]:
n_user = 10000

users = np.arange(0,n_user)
age = np.random.randint(18, 65, size=n_user)
height = np.random.randint(160, 200, size=n_user)


hair_colors = np.random.choice(['blonde', 'brown', 'red'], size=n_user)

df = pd.DataFrame({ 'user':customer_id,'age': age, 'height':height, 'hair':hair_colors})

In [391]:
repetitions = 10
df_exploded = df.loc[np.repeat(df.index.values, repetitions)].reset_index(drop=True)
df_exploded['time_idx'] = np.tile(np.arange(repetitions), len(df))

df = df_exploded

In [393]:
df['brand'] = np.random.choice(['nike','puma','adidas','fila','reebok'], size=n_customers*repetitions)

In [394]:
#TODO: Logik für Target Variable implementieren
df['click'] = 0

previous_brand = 0
current_user = 0 
for index, row in df.iterrows():
    user = row['user']
    brand = row['brand']
    
    if user != current_user:
        previous_brand = 0
        current_user = user
    
    # Check if "nike" appeared twice for the same user
    if brand == 'adidas' and previous_brand == 'nike':
        df.at[index, 'click'] = 1
    
        
    previous_brand = brand

In [ ]:
d

In [467]:
df_test[df_test.user==7].reset_index().drop(columns='index')

,user,age,height,hair,time_idx,brand,click
0,7,39,185,red,0,puma,0
1,7,39,185,red,1,nike,0
2,7,39,185,red,2,nike,0
3,7,39,185,red,3,fila,0
4,7,39,185,red,4,puma,0
5,7,39,185,red,5,fila,0
6,7,39,185,red,6,puma,0
7,7,39,185,red,7,adidas,0
8,7,39,185,red,8,reebok,0
9,7,39,185,red,9,adidas,0


In [395]:
df.head()

,user,age,height,hair,time_idx,brand,click
0,0,36,173,blonde,0,adidas,0
1,0,36,173,blonde,1,nike,0
2,0,36,173,blonde,2,puma,0
3,0,36,173,blonde,3,reebok,0
4,0,36,173,blonde,4,reebok,0


In [396]:
#Outdated
'''
#Effizientere Implementierung

# Create a new column "click" and initialize with 0
df['click'] = 0

# Dictionary to keep track of "nike" count for each user

nike_count = {}

# Iterate over the rows using iterrows()
for index, row in df.iterrows():
    user = row['user']
    brand = row['brand']
    
    # Check if "nike" appeared twice for the same user
    if brand == 'nike' and nike_count.get(user, 0) == 1:
        df.at[index, 'click'] = 1
    
    # Increment the "nike" count for the current user
    if brand == 'nike':
        nike_count[user] = nike_count.get(user, 0) + 1
'''

'\n#Effizientere Implementierung\n\n# Create a new column "click" and initialize with 0\ndf[\'click\'] = 0\n\n# Dictionary to keep track of "nike" count for each user\n\nnike_count = {}\n\n# Iterate over the rows using iterrows()\nfor index, row in df.iterrows():\n    user = row[\'user\']\n    brand = row[\'brand\']\n    \n    # Check if "nike" appeared twice for the same user\n    if brand == \'nike\' and nike_count.get(user, 0) == 1:\n        df.at[index, \'click\'] = 1\n    \n    # Increment the "nike" count for the current user\n    if brand == \'nike\':\n        nike_count[user] = nike_count.get(user, 0) + 1\n'

In [397]:
#Outdated
'''
def determine_target(row):
    
    if row['hair'] == 'brown' and row['brand'] == 'nike':
        return 1
    if row['hair'] == 'blonde' and row['brand'] == 'puma':
        return 1
    if row['hair'] == 'red' and row['brand'] == 'reebok':
        return 1
    return 0
    
    if row['brand'] == 'nike':
        return 1
    return 0
df['click'] = df.apply(determine_target, axis=1)
'''

"\ndef determine_target(row):\n    \n    if row['hair'] == 'brown' and row['brand'] == 'nike':\n        return 1\n    if row['hair'] == 'blonde' and row['brand'] == 'puma':\n        return 1\n    if row['hair'] == 'red' and row['brand'] == 'reebok':\n        return 1\n    return 0\n    \n    if row['brand'] == 'nike':\n        return 1\n    return 0\ndf['click'] = df.apply(determine_target, axis=1)\n"

In [398]:
#Outdated
'''
# Define a function to determine the target value based on the brand
def determine_target(row):
    if row['height'] > 180:
        # Find the previous timestamp where the brand is "nike"
        prev_nike_timestamp = df[(df['customer'] == row['customer']) & (df['timestamp'] < row['timestamp']) & (df['brand'] == 'nike')]['timestamp'].max()
        if prev_nike_timestamp is not None:
            days_diff = row['timestamp'] - prev_nike_timestamp
            if days_diff <= 3 and row['brand'] == 'puma':
                return 1
        # Check if the customer saw nike before and the brand is reebok
        if df[(df['customer'] == row['customer']) & (df['brand'] == 'nike')].shape[0] > 0 and row['brand'] == 'reebok':
            return 1
    return 0

# Apply the function to create the target variable column
df['target'] = df.apply(determine_target, axis=1)
'''

'\n# Define a function to determine the target value based on the brand\ndef determine_target(row):\n    if row[\'height\'] > 180:\n        # Find the previous timestamp where the brand is "nike"\n        prev_nike_timestamp = df[(df[\'customer\'] == row[\'customer\']) & (df[\'timestamp\'] < row[\'timestamp\']) & (df[\'brand\'] == \'nike\')][\'timestamp\'].max()\n        if prev_nike_timestamp is not None:\n            days_diff = row[\'timestamp\'] - prev_nike_timestamp\n            if days_diff <= 3 and row[\'brand\'] == \'puma\':\n                return 1\n        # Check if the customer saw nike before and the brand is reebok\n        if df[(df[\'customer\'] == row[\'customer\']) & (df[\'brand\'] == \'nike\')].shape[0] > 0 and row[\'brand\'] == \'reebok\':\n            return 1\n    return 0\n\n# Apply the function to create the target variable column\ndf[\'target\'] = df.apply(determine_target, axis=1)\n'

In [399]:
#Zähle wie oft welche Brand geklickt worden ist um Ranking bilden zu können. _relative teilt durch Häufigkeit der Brand
df['click'] = df['click'].astype(int)
click_counts = df.groupby('brand')['click'].sum().sort_values(ascending=False)
click_counts_relative = (df.groupby('brand')['click'].sum().sort_index() / df['brand'].value_counts().sort_index()).sort_values(ascending=False)
df['click'] = df['click'].astype(str)

In [400]:
click_counts_relative

brand
adidas    0.180759
fila      0.000000
nike      0.000000
puma      0.000000
reebok    0.000000
dtype: float64

In [401]:
#Caste Types, damit sie in TimeSeriesDataSet verwendet werden können
for c in df.columns:
    df[c] = df[c].astype(str)
    
    if c == 'time_idx' or c == 'user':
        df[c] = df[c].astype(int)

In [447]:
df_test = df[:1000]
df.drop(df_test.index,inplace=True)

In [450]:
training = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= 9],
    time_idx="time_idx",
    target="click",
    group_ids=["user"],
    
    min_encoder_length=9,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=9,
    min_prediction_length=1,
    max_prediction_length=1,
    
    static_categoricals=["age","height","hair"],
    #static_categoricals=["age_level"],
    time_varying_known_categoricals=["brand"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=["click"],

    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

In [451]:
# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 64  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size)

In [404]:
tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=8,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    
    
    #TODO: Loss ändern
    loss=CrossEntropy(),
    optimizer="Ranger"
    # reduce learning rate if no improvement in validation loss after x epochs
    # reduce_on_plateau_patience=1000,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    accelerator="cpu",
    enable_model_summary=True,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
    max_epochs=20,
    logger = logger,
    callbacks=[lr_logger, early_stop_callback],
)

# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | CrossEntropy                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 727   
3  | prescalers                         | ModuleDict                      | 48    
4  | static_variable_selection          | VariableSelectionNetwork        | 584   
5  | encoder_variable_selection         | VariableSelectionNetwork        | 888   
6  | decoder_variable_selection         | VariableSelectionNetwork        | 802   
7  | static_context_variable_selection  | GatedResidualNetwork            | 3

Epoch 0: 100%|█| 156/156 [00:10<00:00, 14.61it/s, v_num=1756, train_loss_step=0.
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:  60%|██████████▏      | 94/157 [00:04<00:03, 20.25it/s]


Validation DataLoader 0:  99%|███████████████▉| 156/157 [00:07<00:00, 20.64it/s]
Epoch 0: 100%|█| 156/156 [00:18<00:00,  8.52it/s, v_num=1756, train_loss_step=0.
Epoch 1: 100%|█| 156/156 [00:10<00:00, 14.86it/s, v_num=1756, train_loss_step=0.
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:  20%|███▎             | 31/157 [00:01<00:06, 19.36it/s]


Validation DataLoader 0:  82%|█████████████   | 128/157 [00:06<00:01, 20.69it/s]


Validation DataLoader 0:  99%|███████████████▉| 156/157 [00:07<00:00, 20.81it/s]
Epoch 1: 100%|█| 156/156 [00:18<00:00,  8.63it/s, v_num=1756, train_loss_step=0.
Epoch 2: 100%|█| 156/156 [00:10<00:00, 14.83it/s, v_num=1756, train_loss_step=8.
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:  41%|███████          | 65/157 [00:03<00:04, 21.33it/s]


...

Validation DataLoader 0:  99%|███████████████▉| 156/157 [00:07<00:00, 21.25it/s]
Epoch 3: 100%|█| 156/156 [00:18<00:00,  8.59it/s, v_num=1756, train_loss_step=2.
Epoch 4: 100%|█| 156/156 [00:10<00:00, 14.77it/s, v_num=1756, train_loss_step=1.
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:  23%|███▉             | 36/157 [00:01<00:05, 21.24it/s]


Validation DataLoader 0:  85%|█████████████▌  | 133/157 [00:06<00:01, 20.63it/s]


Validation DataLoader 0:  99%|███████████████▉| 156/157 [00:07<00:00, 20.61it/s]
Epoch 4: 100%|█| 156/156 [00:18<00:00,  8.56it/s, v_num=1756, train_loss_step=1.
Epoch 4: 100%|█| 156/156 [00:18<00:00,  8.56it/s, v_num=1756, train_loss_step=1.

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|█| 156/156 [00:18<00:00,  8.48it/s, v_num=1756, train_loss_step=1.


In [405]:
def get_tsds(df):
    return TimeSeriesDataSet(
    df[lambda x: x.time_idx <= 9],
    time_idx="time_idx",
    target="click",
    group_ids=["user"],
    
    min_encoder_length=0,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=9,
    min_prediction_length=1,
    max_prediction_length=1,
    
    static_categoricals=["age","height","hair"],
    #static_categoricals=["age_level"],
    time_varying_known_categoricals=["brand"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=["click"],

    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    
    allow_missing_timesteps=True,
    predict_mode=True
)

**Ich verwende die Änderung in beiden Klassenwahrscheinlichkeiten momentan. Man könnte sich aber auch nur die Veränderung für die Klasse 1 anschauen mit .numpy()[1] führt aber trotzdem zu keinen besseren Ergebnissen**

In [442]:
def get_importances(user=7):
    importances = {}
    user_df = df[df['user']==user].reset_index()
    #print(len(user_df))
    tsds = get_tsds(user_df)
    ref = Softmax(dim=2)(tft.predict(tsds, mode="raw", return_x=False).prediction).flatten().numpy()
    feature_values_list = []
    values_list = []
    #print(f"Mit allen Datenpunkten ist die Prediction: {referenz}")
    # Letzte weglassen 0 -> -1 macht den größten Unterschied, ist immer der selbe Wert(?)
    for i in range(len(user_df)-2,0,-1):
        brand = user_df.iloc[i]['brand']
        user_df = user_df.drop(i)
        #print(user_df)
        tsds = get_tsds(user_df)
        pred = Softmax(dim=2)(tft.predict(tsds, mode="raw", return_x=False).prediction).flatten().numpy()
        #diff = round((ref-pred).item(),8)
        diff = np.sum(np.abs(ref-pred))
        print(f"Lasse {brand} weg und erhalte als Prediction {pred}, das sind {diff} Differenz")

        feature_values_list.append(brand)
        values_list.append(diff)
        
        importances[brand] = diff
    return feature_values_list,values_list

In [443]:
users_with_adidas_at_9 = df[df['brand'] == 'adidas'][df['time_idx'] == 9]['user'].unique()
users_with_nike_at_8 = df[df['brand'] == 'nike'][df['time_idx'] == 8]['user'].unique()
users_of_interest = np.intersect1d(users_with_adidas_at_9,users_with_nike_at_8)
print(f"Das sind die User, die für uns von Interesse sind:{users_of_interest[:5]}")

Das sind die User, die für uns von Interesse sind:[ 1 11 18 33 51]


In [444]:
user = 1

In [445]:
df[df['user']==user]

,user,age,height,hair,time_idx,brand,click
10,1,30,169,red,0,reebok,0
11,1,30,169,red,1,reebok,0
12,1,30,169,red,2,reebok,0
13,1,30,169,red,3,nike,0
14,1,30,169,red,4,fila,0
15,1,30,169,red,5,fila,0
16,1,30,169,red,6,adidas,0
17,1,30,169,red,7,fila,0
18,1,30,169,red,8,nike,0
19,1,30,169,red,9,adidas,1


In [446]:
brands,importances = get_importances(user = user)

importances = importances/np.sum(importances)
ranks = []
for b in brands:
    r = int(click_counts_relative.rank(ascending=False)[b])
    ranks.append(r)
importances_df = pd.DataFrame({"brand":brands,"importance":importances,"rank":ranks})
importances_df.sort_values(by="importance",ascending=False)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Lasse nike weg und erhalte als Prediction [9.9998844e-01 1.1516839e-05], das sind 1.9996376037597656 Differenz


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Lasse fila weg und erhalte als Prediction [9.9998701e-01 1.2957901e-05], das sind 1.9996347427368164 Differenz


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Lasse nike weg und erhalte als Prediction [9.9998856e-01 1.1483488e-05], das sind 1.9996378421783447 Differenz


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Lasse reebok weg und erhalte als Prediction [9.99988556e-01 1.14642735e-05], das sind 1.9996378421783447 Differenz
Lasse reebok weg und erhalte als Prediction [9.9998856e-01 1.1441849e-05], das sind 1.9996378421783447 Differenz


,brand,importance,rank
4,fila,0.125,3
5,nike,0.125,3
6,reebok,0.125,3
7,reebok,0.125,3
0,nike,0.125,3
2,adidas,0.125,1
3,fila,0.125,3
1,fila,0.125,3
